<h2>Importing Libraries and Files</h2>

In [1]:
import numpy as np

In [2]:
np.set_printoptions(suppress=True, linewidth=100, precision=2)

In [3]:
loan_data = np.genfromtxt('loan-data.csv', delimiter=";", encoding="cp1252", skip_header=True, autostrip=True)

loan_data

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

In [4]:
np.isnan(loan_data).sum()

88005

In [5]:
temp_fill = np.nanmax(loan_data)+1
temp_mean = np.nanmean(loan_data, axis=0)
temp_mean

/var/folders/dc/__dw74yd6b7bwyhnpw2bmzjh0000gn/T/ipykernel_14510/2436519439.py:2: RuntimeWarning: Mean of empty slice
  temp_mean = np.nanmean(loan_data, axis=0)


array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

In [6]:
temp_stats = np.array([np.nanmin(loan_data, axis=0),
                      temp_mean,
                      np.nanmax(loan_data, axis=0)])

/var/folders/dc/__dw74yd6b7bwyhnpw2bmzjh0000gn/T/ipykernel_14510/613768451.py:1: RuntimeWarning: All-NaN slice encountered
  temp_stats = np.array([np.nanmin(loan_data, axis=0),
/var/folders/dc/__dw74yd6b7bwyhnpw2bmzjh0000gn/T/ipykernel_14510/613768451.py:3: RuntimeWarning: All-NaN slice encountered
  np.nanmax(loan_data, axis=0)])


<h3>Splitting the data into strings and numeric</h3>

In [7]:
column_strings = np.argwhere(np.isnan(temp_mean)).squeeze()
column_numeric = np.argwhere(np.isnan(temp_mean) == False).squeeze()

<h3>Reimporting the dataset</h3>

In [8]:
loan_str_data = np.genfromtxt('loan-data.csv', 
                         delimiter=';', 
                         encoding='cp1252', 
                         skip_header=1,
                         usecols=column_strings, 
                         dtype=str)
loan_str_data

array([['May-15', 'Current', ' 36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', ' 36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', ' 36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [9]:
loan_str_data[:,3]

array(['C', 'A', 'B', ..., 'A', 'D', 'A'], dtype='<U69')

In [47]:
loan_num_data = np.genfromtxt('loan-data.csv', 
                         delimiter=';', 
                         encoding='cp1252', 
                         skip_header=1,
                         filling_values=temp_fill,
                         usecols=column_numeric, 
                         dtype=np.float32)
loan_num_data

array([[48010224.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693260.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432728.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415992.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154152.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055248.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]],
      dtype=float32)

In [11]:
header_full = np.genfromtxt('loan-data.csv', 
                            delimiter=';', 
                            encoding='cp1252',
                            autostrip=True,
                            skip_footer=loan_data.shape[0],
                            dtype=str
                           )
header_full

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [12]:
header_strings = header_full[column_strings]
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [13]:
header_num = header_full[column_numeric]
header_num

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

<h3>Creating Checkpoints</h3>

In [14]:
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header=checkpoint_header, data=checkpoint_data)
    checkpoint_variable = np.load(file_name + ".npz")
    return(checkpoint_variable)

<h3>Data Manipulation<h3>

<h4>Manipulation of loan_str_data</h4>

In [15]:
header_strings[0] = 'issue_date'

In [16]:
loan_str_data[:,0] = np.chararray.strip(loan_str_data[:,0], "-15")

In [17]:
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

In [18]:
for i in range(13):
    loan_str_data[:,0] = np.where(loan_str_data[:,0] == months[i],
                                 i,
                                 loan_str_data[:,0])

In [19]:
np.unique(loan_str_data[:,0])

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

In [20]:
np.unique(loan_str_data[:,1])

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

In [21]:
bad_status = np.array(['', 'Charged Off', 'Default', 'Late (31-120 days)'])

In [22]:
loan_str_data[:,1] = np.where(np.isin(loan_str_data[:,1], bad_status), 0, 1)
np.unique(loan_str_data[:,1])

array(['0', '1'], dtype='<U69')

In [23]:
loan_str_data[:,2]

array([' 36 months', ' 36 months', ' 36 months', ..., ' 36 months', ' 36 months', ' 36 months'],
      dtype='<U69')

In [24]:
loan_str_data[:,2] = np.chararray.strip(np.where(loan_str_data[:,2] == '', '60 months', loan_str_data[:,2]), ' months')
np.unique(loan_str_data[:,2])

array(['36', '60'], dtype='<U69')

In [25]:
header_strings[2] = 'term_months'
header_strings[2]

'term_months'

In [26]:
# Filling sub grades
for i in np.unique(loan_str_data[:,3][1:]):
    loan_str_data[:,4] = np.where(loan_str_data[:,4] == '', 'H1', loan_str_data[:,4])
    
np.unique(loan_str_data[:,4])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

In [28]:
# converting sub grades
keys = list(np.unique(loan_str_data[:,4]))
values = list(range(1, np.unique(loan_str_data[:,4]).shape[0]+1))
dict_sub_grade = dict(zip(keys,values))

dict_sub_grade

{'A1': 1,
 'A2': 2,
 'A3': 3,
 'A4': 4,
 'A5': 5,
 'B1': 6,
 'B2': 7,
 'B3': 8,
 'B4': 9,
 'B5': 10,
 'C1': 11,
 'C2': 12,
 'C3': 13,
 'C4': 14,
 'C5': 15,
 'D1': 16,
 'D2': 17,
 'D3': 18,
 'D4': 19,
 'D5': 20,
 'E1': 21,
 'E2': 22,
 'E3': 23,
 'E4': 24,
 'E5': 25,
 'F1': 26,
 'F2': 27,
 'F3': 28,
 'F4': 29,
 'F5': 30,
 'G1': 31,
 'G2': 32,
 'G3': 33,
 'G4': 34,
 'G5': 35,
 'H1': 36}

In [29]:
for i in np.unique(loan_str_data[:,4]):
    loan_str_data[:,4] = np.where(loan_str_data[:,4] == i, 
                                  dict_sub_grade[i], 
                                  loan_str_data[:,4])

In [30]:
loan_str_data = np.delete(loan_str_data, 3, axis=1)
header_strings = np.delete(header_strings, 3)

In [31]:
loan_str_data[:,3]

array(['13', '5', '10', ..., '5', '17', '4'], dtype='<U69')

In [32]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [33]:
np.unique(loan_str_data[:,4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

In [34]:
verify_status = np.array(['', 'Not Verified'])
loan_str_data[:,4] = np.where(np.isin(loan_str_data[:,4],verify_status),0,1)
np.unique(loan_str_data[:,4])

array(['0', '1'], dtype='<U69')

In [35]:
loan_str_data = np.delete(loan_str_data, 5, axis=1)
header_strings = np.delete(header_strings, 5)

In [36]:
states_west = np.array(['WA','OR', 'CA','NV','ID','MT','WY','UT','CO','AZ','NM','HI','AK']),
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC']),
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH']),
states_east = np.array(['PA','NY', 'NJ', 'CT','MA','VT','NH', 'ME', 'RI'])

In [37]:
loan_str_data[:,5] = np.where(loan_str_data[:,5] =='',0, loan_str_data[:,5])
loan_str_data[:,5] = np.where(np.isin(loan_str_data[:,5], states_west),1,loan_str_data[:,5])
loan_str_data[:,5] = np.where(np.isin(loan_str_data[:,5], states_south),2,loan_str_data[:,5])
loan_str_data[:,5] = np.where(np.isin(loan_str_data[:,5], states_midwest),3,loan_str_data[:,5])
loan_str_data[:,5] = np.where(np.isin(loan_str_data[:,5], states_east),4,loan_str_data[:,5])

In [38]:
np.unique(loan_str_data[:,5])

array(['0', '1', '2', '3', '4'], dtype='<U69')

In [40]:
loan_str_data = loan_str_data.astype(np.int32)

<h3>Checkpoint Strings</h3>

In [41]:
checkpoint_strings = checkpoint('Checkpoint-String', header_strings, loan_str_data)
checkpoint_strings['data']

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]], dtype=int32)

<h4>Manipulation of loan_num_data</h4>

In [42]:
header_num

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [48]:
loan_num_data[:,2] = np.where(loan_num_data[:,2] == temp_fill,
                             temp_stats[0, column_numeric[2]],
                             loan_num_data[:,2])

In [49]:
temp_stats[0, column_numeric[3]]

6.0

In [50]:
for i in [1,3,4,5]:
    loan_num_data[:,i] = np.where(loan_num_data[:,i] == temp_fill,
                            temp_stats[2, column_numeric[i]],
                            loan_num_data[:, i])

In [51]:
loan_num_data

array([[48010224.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693260.  ,    30000.  ,    30000.  ,       28.99,      938.57,     4679.7 ],
       [59432728.  ,    15000.  ,    15000.  ,       28.99,      494.86,     1969.83],
       ...,
       [50415992.  ,    10000.  ,    10000.  ,       28.99,     1372.97,     2185.64],
       [46154152.  ,    35000.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055248.  ,    10000.  ,    10000.  ,       28.99,      309.97,      301.9 ]],
      dtype=float32)

<h3>Exchange Rates</h3>

In [52]:
EUR_USD = np.genfromtxt('EUR-USD.csv', delimiter=',', autostrip=True, skip_header=1, usecols=3)
EUR_USD

array([1.13, 1.12, 1.08, 1.11, 1.1 , 1.12, 1.09, 1.13, 1.13, 1.1 , 1.06, 1.09])

In [54]:
exchange_rate = loan_str_data[:,0]

for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i, EUR_USD[i-1], exchange_rate)
    

exchange_rate = np.where(exchange_rate == 0, np.mean(EUR_USD), exchange_rate)

exchange_rate

array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

In [55]:
exchange_rate = exchange_rate.reshape(10000,1)

In [58]:
loan_num_data = np.hstack((loan_num_data,exchange_rate))

In [67]:
header_num = np.concatenate((header_num, np.array(['exchange_rate'])))

<h3>Converting to EUR from USD</h3>

In [68]:
column_dollar = np.array([1,2,4,5])

In [70]:
for i in column_dollar:
    loan_num_data = np.hstack((loan_num_data, np.reshape(loan_num_data[:,i]/loan_num_data[:,6], (10000,1))))

In [71]:
loan_num_data

array([[48010224.  ,    35000.  ,    35000.  , ...,    31933.3 ,     1081.04,     8624.69],
       [57693260.  ,    30000.  ,    30000.  , ...,    27132.46,      848.86,     4232.39],
       [59432728.  ,    15000.  ,    15000.  , ...,    13326.3 ,      439.64,     1750.04],
       ...,
       [50415992.  ,    10000.  ,    10000.  , ...,     8910.3 ,     1223.36,     1947.47],
       [46154152.  ,    35000.  ,    10000.  , ...,     8997.4 ,      318.78,     2878.63],
       [66055248.  ,    10000.  ,    10000.  , ...,     9145.8 ,      283.49,      276.11]])

<h3>Expanding Header</h3> 

In [72]:
header_added = np.array([column_name + '_EUR' for column_name in header_num[column_dollar]])
header_added

array(['loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U15')

In [73]:
header_num[column_dollar] = np.array([column_name + '_USD' for column_name in header_num[column_dollar]])
header_num

array(['id', 'loan_amnt_USD', 'funded_amnt_USD', 'int_rate', 'installment_USD', 'total_pymnt_USD',
       'exchange_rate'], dtype='<U19')

In [74]:
header_num = np.concatenate((header_num, header_added))
header_num

array(['id', 'loan_amnt_USD', 'funded_amnt_USD', 'int_rate', 'installment_USD', 'total_pymnt_USD',
       'exchange_rate', 'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'],
      dtype='<U19')

In [75]:
column_index_order = [0,1,7,2,8,3,4,9,5,10,6]

In [77]:
header_num = header_num[column_index_order]

In [78]:
loan_num_data = loan_num_data[:,column_index_order]

In [79]:
loan_num_data[:,5] = loan_num_data[:,5]/100
loan_num_data[:,5]

array([0.13, 0.29, 0.29, ..., 0.29, 0.17, 0.29])

<h3>Checkpoint Numeric</h3>

In [80]:
checkpoint_num = checkpoint('Checkpoint-Num', header_num, loan_num_data)
checkpoint_num['header']

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
      dtype='<U19')

<h3>Completing the dataset</h3>

In [81]:
new_loan_data = np.hstack((checkpoint_num['data'], checkpoint_strings['data']))

In [82]:
new_loan_data.shape

(10000, 17)

In [83]:
header_full = np.hstack((checkpoint_num['header'], checkpoint_strings['header']))
header_full

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate',
       'issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')

In [85]:
new_loan_data = new_loan_data[np.argsort(new_loan_data[:,0])]

In [86]:
np.argsort(new_loan_data[:,0])

array([   0,    1,    2, ..., 9997, 9998, 9999])

In [89]:
new_loan_data = np.vstack((header_full, new_loan_data))

In [91]:
np.savetxt('loan-data-preprocessed.csv',new_loan_data, delimiter=',', fmt='%s')